##Call the API with my variables for my research

> List of some states "46 SD", "47 TN","10 DE", "2 AK","5 AR","24 MD","31 NE"

CBSA:

"46 SD",
* "43620 Sioux Falls, SD",

"47 TN"
* "16860 Chattanooga, TN-GA",
* "28940 Knoxville, TN",
* "34980": "Nashville-Davidson--Murfreesboro--Franklin, TN",
* "28700": "Kingsport-Bristol-Bristol, TN-VA",
* "32820": "Memphis, TN-MS-AR",
* "17300": "Clarksville, TN-KY",
* "27740": "Johnson City, TN",
* "17420": "Cleveland, TN",

"10 DE"
* "20100 Dover, DE",
* "428": "Philadelphia-Reading-Camden, PA-NJ-DE-MD",
* "41540": "Salisbury, MD-DE",
*
* 0 city for "2 AK"

"5 AR",
* "22900 Fort Smith, AR-OK",
* "30780 Little Rock-North Little Rock-Conway, AR",
* "22220 Fayetteville-Springdale-Rogers, AR-MO",
* "38220 Pine Bluff, AR",
* "32820 Memphis, TN-MS-AR",
*
"24 MD",
* "25180": "Hagerstown-Martinsburg, MD-WV",
* "428": "Philadelphia-Reading-Camden, PA-NJ-DE-MD",
* "12580": "Baltimore-Columbia-Towson, MD",
* "37980": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",
* "15680": "California-Lexington Park, MD",
* "41540": "Salisbury, MD-DE",

In [3]:
##Constantes
month_dict = {
    1: "jan",
    2: "feb",
    3: "mar",
    4: "apr",
    5: "may",
    6: "jun",
    7: "jul",
    8: "aug",
    9: "sep",
    10: "oct",
    11: "nov",
    12: "dec"
}
# Find the key for 'A MONTH'
def monthTLA(m):
  """
  Will return the key for the 3 letters acronym
  """
  global month_dict
  return next(key for key, value in month_dict.items() if value == m)

#Dict of all states
all_states = requests.get('https://api.census.gov/data/2024/cps/basic/jan/variables.json')
states = json.loads(all_states.text)
states_dict = states['variables']['STATE']['values']['item']
states = [state for state in states_dict.keys()]

#All year of the study

##Variables
stateID = states[1] #47 #range(1-56), without 3, 7, 14, 43
year = 2015 # range(2010-2023)
month = month_dict[10] #range(1-12)

##Calling the API (1st step)

In [65]:
import requests
import json
import numpy as np

# download the Demographics_Native_Country_Of_Mother for Mix-Compostion
key = '&key='+'804d0a1a18d1de70764950c78e2a3a42d3d45e48'
data = 'https://api.census.gov/data/'+ str(year) +'/cps/basic/'+ month + '?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:' + str(stateID) + key
print (data)
try:
  data = requests.get(data)
  print(data.status_code)
except requests.exceptions.RequestException as e:
    # Handle connection errors or HTTP errors
    print("STOP ERROR: Tools for Data Science - Semester Project: An error occurred:\n [[[[", e, "]]]]")
except AttributeError:
  print ("STOP ERROR: Please check your variable name!")

https://api.census.gov/data/2015/cps/basic/oct?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:47&key=804d0a1a18d1de70764950c78e2a3a42d3d45e48
200


##Display a sample (a couple of lines in the top and some at the end)

In [ ]:
##Display the first 100 and last 100 characters
#print (data.text[0:100:],'\n') #TEST
#print (data.text[-100:]) #TEST

## Cleaning the Data
For some reason the data will return tabulate data with 0, witch means no possible match in regards to all the variables.

In [66]:
import pandas as pd
import regex as re

data = data.text
pattern_cleaning = r'\[0,.*?\],?\n?' #remove the data with no tabulate information (tabulate=0)
data = re.sub(pattern_cleaning, '', data)
pattern_cleaning = r'\n' #remove retunr line
data = re.sub(pattern_cleaning, '', data)

##Prep'ing the _columns_ and _data values_ for the _df_ called **mix_composition**

In [67]:
##Separe cols from data (numbers)
pattern = r'.*?(\["tabulate".*?"GTCBSA"\]),([\s\S]*)'
matches = re.search(pattern, data)
data_cols = matches.group(1)
data_vals = matches.group(2)[:-1] #removing the last char "]" at the end.

In [68]:
data_colsList = re.findall(r'"(.*?)"', data_cols) #Transform string_column in List_column

#Transform string_row in List of Lists then Matrix
rows = re.findall(r'\[(.*?)\]', data_vals)
dataM=[]
for row in rows:
  row_list = re.findall(r'-?\d+(?:\.\d+)?', row)
  dataM.append([int(elt) for elt in row_list])

#print (rows[1]) #TEST
#print (dataM[1]) #TEST

##Convert fto df

In [69]:
##Convert the result to a Df **mix_composition**
df = pd.DataFrame(dataM, columns=data_colsList)

##Building the YYYYMM column
from datetime import datetime as dt
df['YYYYMM'] = pd.Series([dt.strptime(str(year)+ '-' +str(month), "%Y-%b") for _ in range(df.shape[0])])
df.head()

,tabulate,state,PEMNTVTY,PEMARITL,PESEX,PRCITSHP,GTCBSA,YYYYMM
0,3214,47,57,6,2,3,17420,2015-10-01
1,2467,47,233,1,1,3,17420,2015-10-01
2,1889,47,57,1,1,3,17420,2015-10-01
3,2422,47,303,2,2,5,17420,2015-10-01
4,2612,47,301,1,2,5,17420,2015-10-01


In [70]:
##How many cities represented for the sigle stateID?
#Remark: for 47-TN I got all the cities listed as predicted [17420 28700 27740 32820 34980 17300 16860 28940]
print(pd.unique(df['GTCBSA']))

[17420 28700 27740 32820 34980 17300 16860 28940]


In [160]:
url = 'https://api.census.gov/data/2023/cps/basic/jan?get=PWSSWGT.PEMNTVTY,PEMARITL,GTCBSA&for=state:31'
def checkVariable(url):
    """
    this function will check if a set of variables are declared on the Census API. Using the an url as input. Vars are in ALL_CAPS
    before 2024, City variable was in variable 'GTCBSA'
    starting on 2024, City variable was in variable 'CBSA'
    """
    matches = re.findall(r'([A-Z]+)', url)
    cps_vars = json.loads(requests.get('https://api.census.gov/data/2023/cps/basic/jan/variables.json').text)
    res = dict()
    for usedVar in matches:
      res[usedVar] = True if usedVar in cps_vars['variables'].keys() else False
    res['status'] = all([status for status in res.values()])
    return res

checkVariable(url)

{'PWSSWGT': True,
 'PEMNTVTY': True,
 'PEMARITL': True,
 'GTCBSA': True,
 'status': True}

#class Creation
Now we have all we need for our analysis, let's try to reorganize what we build so far into a class called _MixComp_

In [76]:
#Class API
class API:
  """This generic class API will handle at once all request to the CPS api
  varsUrl (variable): given a year, it will return the url for all variables
  allVars_dict (variable): a dict of all variables for each 'year' (key)
  examples:
   - API(year=2010).varsUrl * url for all variables in 2010
   - API().allVars_dict[2023] * to se all the variables (dict) in 2023
   - 'GTCBSA' in API().allVars_dict[2024].keys() * is <GTCBSA> in 2024?
   - API().allState_dict[2021] * all stateID:description in year 2021"""

  def __init__(self) -> None:
    pass

  def get(self, year_search=2010):
    self.varsUrl = 'https://api.census.gov/data/'+ str(year_search) +'/cps/basic/jan/variables.json'

    #all_years = range(2022, 2024)
    all_years = range(2010, 1+dt.now().year)
    self.allVars_dict = {year_: year_vars for year_, year_vars in \
                      zip(all_years, [json.loads(requests.get('https://api.census.gov/data/'+ str(year_) +'/cps/basic/jan/variables.json').text)['variables'] for year_ in all_years])}

#    self.allStates_dict = {_year: state_vars for _year, state_vars in \
#                  zip(all_years, [ self.allVars_dict[_year]['GTCBSA']['values']['item'] for _year in all_years])}

  def store(self):
    pass

  def getVarsNameFromUrl(self, url):
    return re.findall(r'([A-Z]+)', url)

api=API()
api.get()
api.allVars_dict.keys()
a = 'Y&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:47&key=804d0a1a18d1de70764950c78e2a3a42d3d45e48'
API().getVarsNameFromUrl(a)

['Y', 'PEMARITL', 'PESEX', 'PRCITSHP', 'GTCBSA']

In [61]:
api.allVars_dict[2021].keys()

dict_keys(['for', 'in', 'ucgid', 'PEEDUCA', 'PUBUS1', 'PRCOW1', 'HETELHHD', 'PRCOW2', 'PEERNLAB', 'PRTAGE', 'PRERELG', 'GTCBSAST', 'PXNMEMP2', 'PXNMEMP1', 'GTINDVPC', 'PEDISDRS', 'PECOHAB', 'PUCHINHH', 'PTERN', 'PXERNRT', 'PXLKM1', 'PXLKFTO', 'PECERT2', 'PECERT1', 'PECERT3', 'PEIO2ICD', 'PXFNTVTY', 'PXIO1ICD', 'PRPERTYP', 'PXERNHRO', 'PXERNH1O', 'PRMJIND2', 'PRMJIND1', 'PXLAYAVL', 'PRUNTYPE', 'PEJHWANT', 'HXTELAVL', 'PRCOWPG', 'PXPAR2', 'PUDIS2', 'PXPAR1', 'PUDIS1', 'PRINUYER', 'PXRET1', 'GTCBSASZ', 'PXERNHRY', 'PRDASIAN', 'PXCERT1', 'PXCERT3', 'PXCERT2', 'PEERNCOV', 'PUSLFPRX', 'HRHTYPE', 'PRAGNA', 'PEERNPER', 'PXCOHAB', 'PRHERNAL', 'PELKFTO', 'PUDWCK5', 'PUDWCK4', 'HUPRSCNT', 'PUDWCK1', 'PXABSRSN', 'PXLKDUR', 'PUDWCK3', 'PUDWCK2', 'PUHRCK12', 'PULKPS1', 'PULKPS2', 'PEERNUOT', 'PULKPS3', 'PULKPS4', 'PULKPS5', 'PRHRUSL', 'PULKPS6', 'PREXPLF', 'PXIO2COW', 'PULKAVR', 'HETELAVL', 'PRNLFSCH', 'PEDWLKWK', 'PRCHLD', 'GTCBSA', 'PULAYAVR', 'PUDIS', 'PXRACE1', 'PEDISPHY', 'PXERNWKP', 'PRTFAGE',

In [15]:
import requests
import pandas as pd
import regex as re
import time
from datetime import datetime as dt
import json

In [83]:
#Decorator to track API response time
def track_response_time(func):
    def wrapper(self, *args, **kwargs):
        start_time = time.time()
        result = func(self, *args, **kwargs)
        end_time = time.time()
        response_time = end_time - start_time
        self.response_time = response_time  # Store in the class
        print(f"Function: '{func.__name__}()'. Census API query took: {response_time} seconds")
        return result
    return wrapper


#Class for unit-request (specific State, specific Year, specific Month)
class MixComp:
  """
This class will pull the Mix composition about the US population from one state.
Code	    Label
tabulate  Counts of instances
PEMNTVTY	Demographics-native country of mother
PEMARITL	Demographics-marital status
PESEX	    Demographics-sex
PRCITSHP	Demographics-United States citizenship group
GTCBSA	  Demographics-city level
STATE	    FIPS STATE Code
GTCBSA	  Demographics-city level

  """
  def __init__(self, stateID, year, month):
      key = '&key='+'804d0a1a18d1de70764950c78e2a3a42d3d45e48'
      self.url = 'https://api.census.gov/data/'+ str(year) +'/cps/basic/'+ month + '?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:' + str(stateID) + key
      self.stateID = stateID
      self.year = year
      self.month = month
      self.response_time = 0 # how long the API took to run on Census CPS
      self.dataAPI = None
      self.dataText = None
      self.data4Df = []
      self.df = pd.DataFrame(None) #Will contain the final df(dataframe)

  def __str__(self):
      return f"""
      Mix Composition of the US population: stateID:={self.stateID}, month={self.month}, year={self.year}
      Dataframe head:
         {self.df.head(3)}"""

  def __repr__(self):
      return f"MixComp({self.stateID}, {self.year}, {self.month})"

  def __len__(self):
      return self.df.shape[0]

  def __add__(self, other):
      return pd.concat(self.df, other.df, axis=1)

  # to convert in class
  def checkVariable(self, year, url):
      """
      this function will check if a set of variables are declared on the Census API. Using the an url as input. Vars are in ALL_CAPS
      """
      api=API()
      api.get()
      matches = api.getVarsNameFromUrl(url)
      cps_vars = api.allVars_dict[year].keys()
      res = dict()
      for var in matches:
        res[var] = True if var in cps_vars else False
      res['status'] = all([status for status in res.values()])
      return res

  @track_response_time
  def get(self):

      #This section will validate all variables and replace the city's with the right one if year>=2024
      check = self.checkVariable(self.year, self.url)
      print (f"checkVariable() = {check}")
      if not check['status']:
        print(f"BEFORE - Variables = {check}")
        if (self.year>=2024) and ('CBSA' not in check.keys()): #before 2024, City variable in 'GTCBSA', starting on 2024, City variable was 'CBSA'
          self.url = re.sub('GTCBSA', 'CBSA', self.url)
          print(f"AFTER -  Variables = {self.checkVariable(self.year, self.url)}\nNew URL: {self.url}")
        else:
          raise(f"Please check all your Census variables. Unable to find some in the API call. Variables = {check}")

      try:
        # download the Demographics_Native_Country_Of_Mother for Mix-Compostion
        self.dataAPI = requests.get(self.url)
        return dict({'response time in secs': round(self.response_time,0), 'status': True if 200==self.dataAPI.status_code else False, 'code':self.dataAPI.status_code, 'comment': 'url:' + self.url})
        None
      except requests.exceptions.RequestException as e:
          # Handle connection errors or HTTP errors
          raise("STOP ERROR: Tools for Data Science - Semester Project: An error occurred:\n [[[[", e, "]]]]")

      except AttributeError:
          raise("STOP ERROR: Please check your variable name!")

  def cleaning(self):
      self.dataText = self.dataAPI.text
      pattern_cleaning = r'\[0,.*?\],?\n?' #remove the data with no tabulate information (tabulate=0)
      self.dataText = re.sub(pattern_cleaning, '', self.dataText)
      pattern_cleaning = r'\n' #remove retunr line
      self.dataText = re.sub(pattern_cleaning, '', self.dataText)

  def preping(self):
      ##Separe cols from data (numbers)
      pattern = r'.*?(\["tabulate".*?"GTCBSA"\]),([\s\S]*)'
      matches = re.search(pattern, self.dataText)
      data_cols = matches.group(1)
      data_vals = matches.group(2)[:-1] #removing the last char "]" at the end.

      data_colsList = re.findall(r'"(.*?)"', data_cols) #Transform string_column in List_column

      #Transform string_row in List of Lists then Matrix
      rows = re.findall(r'\[(.*?)\]', data_vals)
      for row in rows:
        row_list = re.findall(r'-?\d+(?:\.\d+)?', row)
        self.data4Df.append([int(elt) for elt in row_list])

      #print (rows[1]) #TEST
      #print (dataM[1]) #TEST

  def buildingDf(self):
      """
      ##Definition of variables:
      * tabulate: represents the information related to population counts.
      * state: column provides information about the state
      1. PEMNTVTY Demographics-native country of mother
      2. PEMARITL Demographics-marital status
      3. PESEX Demographics-sex
      4. PRCITSHP Demographics-United States citizenship group
      5. GTCBSA Demographics-city level
      We will understand the population composition by state about how mixed it is based on the native country of mothers,
      marital status distribution, gender demographics, citizenship status, at the city-level characteristics.
      This data can be used to uncover trends, patterns, and insights into the social and cultural fabric of the US population.
      """
      ##Convert the result to the Df **mix_composition**
      self.df = pd.DataFrame(self.data4Df, columns=data_colsList)

      ##Building the YYYYMM column
      self.df['YYYYMM'] = pd.Series([dt.strptime(str(self.year)+ '-' +str(self.month), "%Y-%b") for _ in range(self.df.shape[0])])

  def to_csv(self, path='./'):
      try:
        r = self.df.to_csv( str(self.year)+ '-' +str(self.month) +'-MixComp.csv', index=False)
        return r
      except FileNotFoundError:
        raise ('Check the path {path} of the file')

  def main(self):
      self.get()
      self.cleaning()
      self.preping()
      self.buildingDf()
      self.to_csv()


##Let's test the Class MixComp

In [82]:
stateID = states[1] #47 #range(1-56), without 3, 7, 14, 43
year = 2024 # range(2010-2023)
month = month_dict[10] #range(1-12)

data = MixComp(stateID, year, month)
print(data.url)

r = data.get()
print(r)

https://api.census.gov/data/2024/cps/basic/oct?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:47&key=804d0a1a18d1de70764950c78e2a3a42d3d45e48
checkVariable() = {'PWSSWGT': True, 'PEMNTVTY': True, 'PEMARITL': True, 'PESEX': True, 'PRCITSHP': True, 'GTCBSA': False, 'status': False}
BEFORE - Variables = {'PWSSWGT': True, 'PEMNTVTY': True, 'PEMARITL': True, 'PESEX': True, 'PRCITSHP': True, 'GTCBSA': False, 'status': False}
AFTER -  Variables = {'PWSSWGT': True, 'PEMNTVTY': True, 'PEMARITL': True, 'PESEX': True, 'PRCITSHP': True, 'CBSA': True, 'status': True}
New URL: https://api.census.gov/data/2024/cps/basic/oct?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+CBSA&for=state:47&key=804d0a1a18d1de70764950c78e2a3a42d3d45e48
Function: 'pull()'. Census API query took: 14.991904973983765 seconds
None


In [100]:
data.cleaning()

In [101]:
data.preping()

In [102]:
data.buildingDf()

In [103]:
data.df

,tabulate,state,PEMNTVTY,PEMARITL,PESEX,PRCITSHP,GTCBSA,YYYYMM
0,3214,47,57,6,2,3,17420,2015-10-01
1,2467,47,233,1,1,3,17420,2015-10-01
2,1889,47,57,1,1,3,17420,2015-10-01
3,2422,47,303,2,2,5,17420,2015-10-01
4,2612,47,301,1,2,5,17420,2015-10-01
...,...,...,...,...,...,...,...,...
303,192539,47,57,1,1,1,28940,2015-10-01
304,2580,47,57,3,1,1,28940,2015-10-01
305,88719,47,57,-1,1,1,28940,2015-10-01
306,107142,47,57,6,1,1,28940,2015-10-01


<Response [200]>

{'46': 'SD',
 '47': 'TN',
 '10': 'DE',
 '2': 'AK',
 '5': 'AR',
 '24': 'MD',
 '31': 'NE',
 '44': 'RI',
 '48': 'TX',
 '11': 'DC',
 '16': 'ID',
 '4': 'AZ',
 '6': 'CA',
 '19': 'IA',
 '22': 'LA',
 '26': 'MI',
 '45': 'SC',
 '50': 'VT',
 '51': 'VA',
 '1': 'AL',
 '8': 'CO',
 '32': 'NV',
 '37': 'NC',
 '49': 'UT',
 '54': 'WV',
 '55': 'WI',
 '13': 'GA',
 '15': 'HI',
 '17': 'IL',
 '21': 'KY',
 '34': 'NJ',
 '38': 'ND',
 '39': 'OH',
 '41': 'OR',
 '30': 'MT',
 '40': 'OK',
 '56': 'WY',
 '28': 'MS',
 '36': 'NY',
 '42': 'PA',
 '53': 'WA',
 '12': 'FL',
 '18': 'IN',
 '9': 'CT',
 '20': 'KS',
 '23': 'ME',
 '25': 'MA',
 '27': 'MN',
 '29': 'MO',
 '33': 'NH',
 '35': 'NM'}